In [21]:
using CUDA
device!(2)
# 0 indexed

include("ese156_tools.jl")
using RadiativeTransfer
using RadiativeTransfer.Architectures
using RadiativeTransfer.CrossSection

# ask rupesh about 32 float
# 25hpa spacing, 200-400 KM (every 5 K)

In [22]:
# To change things up, choose lat/lon of your birthplace (or any place you like!)
file_folder = "files"
# Locate articafts, see https://julialang.github.io/Pkg.jl/v1/artifacts/
# file = joinpath(file_folder, "MERRA300.prod.assim.inst6_3d_ana_Nv.20150613.hdf.nc4")

file = "/home/cchristo/proj_christian/rad_transfer_julia/files/MERRA300.prod.assim.inst6_3d_ana_Nv.20150613.hdf.nc4"
timeIndex = 2 # There is 00, 06, 12 and 18 in UTC, i.e. 6 hourly data stacked together

# What latitude do we want?
myLat = 34.05;
myLon = -118.24;

In [23]:
profile = read_atmos_profile(file, myLat, myLon, timeIndex);

ds["T"] = T (540 × 361 × 72 × 4)
  Datatype:    Float32
  Dimensions:  XDim × YDim × Height × TIME
  Attributes:
   _FillValue           = 1.0e15
   long_name            = Air temperature
   standard_name        = air_temperature
   units                = K
   scale_factor         = 1.0
   add_offset           = 0.0
   missing_value        = 1.0e15
   fmissing_value       = 1.0e15
   vmin                 = -1.0e30
   vmax                 = 1.0e30
   valid_range          = Float32[-1.0f30, 1.0f30]
   coordinates          = TIME Height YDim XDim
   fullpath             = /EOSGRID/Data Fields/T



In [24]:
# Minimum wavenumber
ν_min  = 400.0
# Maximum wavenumber
ν_max = 2100.0

co2_par = CrossSection.read_hitran(joinpath(file_folder, "hitran_molec_id_2_CO2.tar"), mol=2, iso=1, ν_min=ν_min, ν_max=ν_max);
ch4_par = CrossSection.read_hitran(joinpath(file_folder, "hitran_molec_id_6_CH4.tar"), mol=6, iso=1, ν_min=ν_min, ν_max=ν_max);
h2o_par = CrossSection.read_hitran(joinpath(file_folder, "hitran_molec_id_1_H2O.tar"), mol=1, iso=1, ν_min=ν_min, ν_max=ν_max);

In [34]:
# GPU
co2_voigt   = make_hitran_model(co2_par, Voigt(), wing_cutoff=10, CEF=HumlicekWeidemann32SDErrorFunction(), architecture=CrossSection.GPU())
h2o_voigt   = make_hitran_model(h2o_par, Voigt(), wing_cutoff=10, CEF=HumlicekWeidemann32SDErrorFunction(), architecture=CrossSection.GPU())
ch4_voigt   = make_hitran_model(ch4_par, Voigt(), wing_cutoff=10, CEF=HumlicekWeidemann32SDErrorFunction(), architecture=CrossSection.GPU())

hitran_array = [co2_voigt, h2o_voigt, ch4_voigt];

In [54]:
res = 0.01
ν = ν_min:res:ν_max
# σ_matrix = compute_profile_crossSections(profile, hitran_array , ν);

# New


In [44]:
pressures = 1250:10:150 #pressure can be relaxed 
temperatures = 180:5:350

180:5:350

In [37]:
# co2_par
# 2:32

400.0:0.01:2100.0

In [60]:
# Create the Interpolation Model
pressures = 150:10:1200
temperatures = 180:5:335
# pressures = 250:250:1250
# temperatures = 100:75:400 

# interp_model_co2 = make_interpolation_model(co2_par, Voigt(), ν, pressures, 
#                                         temperatures, CEF=HumlicekWeidemann32SDErrorFunction())
# #                                         architecture=CrossSection.GPU()) 

interp_model_h2o = make_interpolation_model(h2o_par, Voigt(), ν, pressures, 
                                        temperatures, CEF=HumlicekWeidemann32SDErrorFunction(),
                                        architecture=CrossSection.GPU())

interp_model_ch4 = make_interpolation_model(ch4_par, Voigt(), ν, pressures, 
                                        temperatures, CEF=HumlicekWeidemann32SDErrorFunction(),
                                        architecture=CrossSection.GPU())


Computing Cross Sections for Interpolation...100%|██████| Time: 0:38:32
Computing Cross Sections for Interpolation...100%|██████| Time: 2:56:40


InterpolationModel
  itp: Interpolations.BSplineInterpolation{Float64,3,OffsetArrays.OffsetArray{Float64,3,Array{Float64,3}},BSpline{Cubic{Line{OnGrid}}},Tuple{Base.OneTo{Int64},Base.OneTo{Int64},Base.OneTo{Int64}}}
  mol: Int64 6
  iso: Int64 1
  broadening: Voigt Voigt()
  ν_grid: StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
  p_grid: StepRange{Int64,Int64}
  t_grid: StepRange{Int64,Int64}
  wing_cutoff: Int64 40
  vmr: Int64 0
  CEF: HumlicekWeidemann32SDErrorFunction HumlicekWeidemann32SDErrorFunction()
  architecture: GPU GPU()


In [61]:
# start 5:04

save_interpolation_model(interp_model_h2o, "/net/fluo/data2/groupMembers/cchristo/interp_models/test/h2o_p10_t_5")
save_interpolation_model(interp_model_ch4, "/net/fluo/data2/groupMembers/cchristo/interp_models/test/ch4_p10_t_5")

In [59]:
# save_interpolation_model(interp_model, "/net/fluo/data2/groupMembers/cchristo/interp_models/test/co2_p10_t_5")

In [32]:
pressures = 250:250:1250
temperatures = 100:75:400 
# Wavelength grid
ν_grid = 6000:0.01:6400

6000.0:0.01:6400.0